# Filtragem Colaborativa por Item

## Introdução

A filtragem colaborativa faz recomendações com base em padrões de comportamento passado de vários usuários, sem necessitar de qualquer informação adicional sobre os itens ou usuários. A ideia básica da filtragem colaborativa é criar uma matriz usuário-item. O conjunto de dados pode ser representado como uma matriz onde as linhas correspondem aos usuários e as colunas aos filmes. 

Matrix | Inception | Titanic | Star Wars | The Godfather
--------|-----------|---------|-----------|--------------
Alice    |     5     |    3    |     4     |      0
Bob      |     4     |    0    |     5     |      3
Carol    |     3     |    5    |     4     |      4
Dave     |     0     |    2    |     0     |      5
Eve      |     2     |    5    |     0     |      4

## Ingestão dos dados

**Bibliotecas**

In [32]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np

In [33]:
# Dados de entrada
data = {
    'Inception': [5, 4, 3, np.nan, 2],
    'Titanic': [3, np.nan, 5, 2, 5],
    'Star Wars': [4, 5, 4, np.nan, np.nan],
    'The Godfather': [np.nan, 3, 4, 5, 4]
}

# Cria um dataframe do pandas
df = pd.DataFrame(data, index=['Alice', 'Bob', 'Carol', 'Dave', 'Eve'])
df.head(2)

,Inception,Titanic,Star Wars,The Godfather
Alice,5.0,3.0,4.0,NaN
Bob,4.0,NaN,5.0,3.0


Para fazer a Filtragem Colaborativa por Item, necessitamos transpor essa matriz de forma que os filmes sejam as linhas e os usuários, as colunas.

In [34]:
# Transpõe o dataframe
df = df.transpose()
df

,Alice,Bob,Carol,Dave,Eve
Inception,5.0,4.0,3.0,NaN,2.0
Titanic,3.0,NaN,5.0,2.0,5.0
Star Wars,4.0,5.0,4.0,NaN,NaN
The Godfather,NaN,3.0,4.0,5.0,4.0


## Preparação dos Dados

In [8]:
# Substituir NaN por 0
df_filled = df.fillna(0)
df_filled

,Alice,Bob,Carol,Dave,Eve
Inception,5.0,4.0,3.0,0.0,2.0
Titanic,3.0,0.0,5.0,2.0,5.0
Star Wars,4.0,5.0,4.0,0.0,0.0
The Godfather,0.0,3.0,4.0,5.0,4.0


Normalização das avaliações. Essa normalização ajuda a lidar com o viés de que diferentes usuários podem ter diferentes escalas de classificação. Por exemplo, um usuário pode ser geralmente mais crítico e dar classificações mais baixas, enquanto outro pode dar classificações mais altas. Normalizar os dados dessa maneira permite que o sistema de recomendação lide melhor com essas diferenças de escala.

In [10]:
# Normalizar os dados subtraindo a média de cada usuário
normalized_df = df_filled.sub(df_filled.mean(axis=1), axis=0)
normalized_df

,Alice,Bob,Carol,Dave,Eve
Inception,2.2,1.2,0.2,-2.8,-0.8
Titanic,0.0,-3.0,2.0,-1.0,2.0
Star Wars,1.4,2.4,1.4,-2.6,-2.6
The Godfather,-3.2,-0.2,0.8,1.8,0.8


## Desenvolvimento do Modelo

### KNN

In [22]:
# Usar o algoritmo KNN com a métrica de similaridade do cosseno
knn = NearestNeighbors(metric='cosine', n_neighbors=3, n_jobs=-1)

# Ajustar o modelo com os dados normalizados
knn.fit(normalized_df)

# Calcular as distâncias e os índices dos vizinhos mais próximos para todos os usuários
distances, indices = knn.kneighbors(normalized_df)

In [23]:
print(distances)

[[2.22044605e-16 1.58120861e-01 1.12253577e+00]
 [0.00000000e+00 8.77464230e-01 1.12253577e+00]
 [1.11022302e-16 1.58120861e-01 1.34254513e+00]
 [1.11022302e-16 8.77464230e-01 1.57204608e+00]]


In [24]:
print(indices)

[[0 2 1]
 [1 3 0]
 [2 0 1]
 [3 1 2]]


### Interpretando os resultados

#### `distances`

A matriz `distances` contém os valores da distância do cosseno entre cada ponto e seus vizinhos mais próximos. As distâncias são medidas em um intervalo de 0 a 2, onde 0 indica similaridade total (ou seja, o ângulo entre os dois vetores é 0 graus, então eles são idênticos) e 2 indica dissimilaridade total (ou seja, os vetores são opostos). Aqui, um valor muito próximo de zero, como `2.22044605e-16`, é numericamente equivalente a zero devido à precisão do ponto flutuante e indica que o ponto é o mais próximo possível de si mesmo.

Cada linha na matriz `distances` corresponde a um filme, e cada coluna é a distância desse ponto aos seus `k` vizinhos mais próximos. Escolhemos `k=3`, portanto, há três colunas.

Por exemplo, para o primeiro filme (`Inception`), a distância para si mesmo é ~0 (a precisão do ponto flutuante faz aparecer como `2.22044605e-16`), para o segundo vizinho mais próximo é `0.158120861` e para o terceiro é `1.12253577`.

#### `indices`

A matriz `indices` contém os índices dos vizinhos mais próximos para cada ponto. Esses índices correspondem à posição original dos filmes na matriz `df` transposta.

Cada linha em `indices` representa os índices dos vizinhos mais próximos de um filme. A primeira coluna é o índice do próprio filme (já que a distância mais curta para um ponto é sempre para si mesmo), e as colunas subsequentes são os índices dos vizinhos mais próximos.

No exemplo, para o primeiro filme (`Inception`), os índices dos vizinhos mais próximos são:
- `0` que é ele mesmo (`Inception`),
- `2` que é `Star Wars`,
- `1` que é `Titanic`.

Isso significa que, com base nas avaliações que foram normalizadas e segundo a métrica de similaridade do cosseno, `Star Wars` é o filme mais similar a `Inception`, seguido por `Titanic`.

As distâncias e índices são consistentes entre si, ou seja, a distância na matriz `distances` na posição [0, 1] é a distância do filme `Inception` para o filme `Star Wars`, cujo índice é encontrado na matriz `indices` na posição [0, 1].

Quando você usa essas informações para recomendar filmes, você olharia para os vizinhos mais próximos de um filme que um usuário gosta para encontrar outros filmes que eles também podem gostar.

## Função de Recomendação

### Passo-a-passo

In [16]:
user_index = 0 # Alice
item_index = 3 # The Godfather

In [17]:
neighbor_indices = indices[item_index, 1:]  # Ignora o próprio item. A primeira coluna é o próprio filme
neighbor_indices

array([1, 2])

In [26]:
# Obtém as classificações que os usuários deram para os itens vizinhos. 
neighbor_ratings = normalized_df.iloc[neighbor_indices, user_index]
neighbor_ratings

Titanic      0.0
Star Wars    1.4
Name: Alice, dtype: float64

> A **similaridade** do cosseno, por definição, varia de -1 a 1. Um valor de 1 significa que dois vetores estão na mesma direção, um valor de 0 significa que são ortogonais (independentes), e um valor de -1 significa que estão em direções opostas. No contexto de sistemas de recomendação e outras aplicações de aprendizado de máquina, a similaridade do cosseno é muitas vezes usada para medir a similaridade entre dois vetores de características, com 1 sendo a similaridade máxima e -1 a mínima.

> Por outro lado, a **distância** do cosseno, que é uma função da similaridade do cosseno, varia de 0 a 2. A distância do cosseno é calculada como `1 - similaridade_do_cosseno`, transformando a escala de -1 a 1 para uma escala de 0 a 2. Um valor de 0 indica que os vetores são idênticos, enquanto um valor de 2 indica que eles são completamente diferentes. A distância do cosseno é usada porque os algoritmos de aprendizado de máquina frequentemente requerem uma função de distância que seja não negativa e que obedeça a desigualdade triangular.

> No nosso caso, como estamos trabalhando com a função `NearestNeighbors` do `scikit-learn`, que busca os vizinhos mais próximos baseando-se em uma função de distância, daí a razão dos valores variarem de 0 a 2. 

In [27]:
"""
`np.sum(1 - distances[item_index, 1:])`:
   - `distances[item_index, 1:]` obtém todas as distâncias do cosseno dos vizinhos mais próximos para um item específico, excluindo a primeira distância, que é a distância do item para si mesmo (sempre 0 após a subtração).
   - `1 - distances[item_index, 1:]` converte distâncias do cosseno em similaridade do cosseno, já que a distância do cosseno é `1 - similaridade`.
   - `np.sum(1 - distances[item_index, 1:])` soma essas similaridades para usar como pesos na média ponderada.
"""
sum_weights = np.sum(1 - distances[item_index, 1:])  # Subtrai de 1 porque a distância do cosseno é 1 - similaridade
sum_weights

-0.4495103112546899

In [28]:
"""
`weighted_sum = np.sum((1 - distances[item_index, 1:]) * neighbor_ratings)`:
   - `neighbor_ratings` é o vetor que contém as classificações dos vizinhos mais próximos para o item em questão.
   - `(1 - distances[item_index, 1:]) * neighbor_ratings` calcula o produto da similaridade de cada vizinho com sua respectiva classificação, que é uma forma de ponderar as classificações com base em quão semelhantes esses vizinhos são ao item de interesse.
   - `np.sum((1 - distances[item_index, 1:]) * neighbor_ratings)` soma esses produtos ponderados para obter uma soma ponderada total das classificações.
"""
weighted_sum = np.sum((1 - distances[item_index, 1:]) * neighbor_ratings)
weighted_sum

-0.800864514245121

In [29]:
"""
`predicted_rating = weighted_sum / sum_weights`:
   - Esta é uma média ponderada das classificações dos vizinhos, onde as ponderações são baseadas nas similaridades do cosseno (convertidas de distâncias do cosseno).
   - `weighted_sum / sum_weights` divide a soma ponderada pela soma dos pesos (similaridades), para calcular a classificação prevista para o item.
"""
predicted_rating = weighted_sum / sum_weights
predicted_rating

1.7816376937154526

### Colocando tudo em uma função

In [ ]:
# A função para prever a classificação precisa ser ajustada para a filtragem colaborativa por item
def predict_rating(item_index, user_index, data, indices):
    # Seleciona os índices dos itens mais próximos (vizinhos) para o item alvo.
    neighbor_indices = indices[item_index, 1:]  # Ignora o próprio item

    # Obtém as classificações que os usuários deram para os itens vizinhos. 
    neighbor_ratings = data.iloc[neighbor_indices, user_index]

    # Calcular a média ponderada das classificações dos vizinhos
    # As distâncias servem como pesos inversos para a média ponderada
    sum_weights = np.sum(1 - distances[item_index, 1:])  # Subtrai de 1 porque a distância do cosseno é 1 - similaridade
    weighted_sum = np.sum((1 - distances[item_index, 1:]) * neighbor_ratings)
    
    if sum_weights == 0:  # Evita divisão por zero
        predicted_rating = 0
    else:
        predicted_rating = weighted_sum / sum_weights

    return predicted_rating


user_index = 0 # Alice
item_index = 3 # The Godfather

# Prever a classificação para Alice (índice 0) para 'The Godfather' (índice 3)
predicted_rating_for_alice = predict_rating(item_index, user_index, df_filled, indices)

predicted_rating_for_alice